In [7]:
import os
import soundfile as sf
import librosa
import numpy as np
import torch
import torch.nn as nn
import torchvision.models as models
import matplotlib.pyplot as plt

Configurations

In [ ]:
SAMPLE_RATE = 16000
DURATION = 3
AUDIO_DIR_PATH = "audio" 

Load audio and load it into raw waveforms segments

In [9]:
for audio_file_name in os.listdir(AUDIO_DIR_PATH):    
    waveform, sr = librosa.load(os.path.join(AUDIO_DIR_PATH, audio_file_name), sr=SAMPLE_RATE)
    total_samples = DURATION * SAMPLE_RATE 
    segments = []
    for i in range(0, len(waveform) - total_samples + 1, total_samples):
        segments.append(waveform[i:i + total_samples]) # last incomplete segment will be excluded

Spectograms Visualization

Prosodic Features

In [ ]:
for segment in segments:

    # 1. Pitch (using YIN)
    pitch = librosa.yin(segment, fmin=50, fmax=300, sr=sr)
    # 2. Energy (RMS)
    energy = librosa.feature.rms(y=segment)[0]
    # 3. Speaking rate (rough proxy using tempo)
    tempo, _ = librosa.beat.beat_track(y=segment, sr=sr)
    # Frame-align pitch & energy (zero-pad to same length)
    length = min(len(pitch), len(energy))
    pitch = pitch[:length]
    energy = energy[:length]
    # Combine frame-wise
    features = np.stack([pitch, energy], axis=1)  # [T x 2]

Model Loading